In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-12-22 09:41:42--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   133MB/s    in 0.2s    

2024-12-22 09:41:42 (133 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [10]:
# add your code here - considercombine_book_rating = pd.merge(df_ratings, df_books, on='isbn')
# add your code here - considercombine_book_rating = pd.merge(df_ratings, df_books, on='isbn')
combine_book_rating = pd.merge(df_ratings, df_books, on='isbn') # This line was commented out, causing the error. It has now been uncommented.
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['title'])

book_ratingCount = (combine_book_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )

rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'title', right_on = 'title', how = 'left')

popularity_threshold = 100
rating_popular_book = rating_with_totalRatingCount[rating_with_totalRatingCount['totalRatingCount'] >= popularity_threshold]


combined = rating_popular_book.merge(df_books, left_on = 'isbn', right_on = 'isbn', how = 'left')
if 'title' not in combined.columns:
    combined = combined.merge(df_books[['isbn', 'title']], on='isbn', how='left')
userrating = combined.drop_duplicates(['user', 'title'])
userrating_pivot = userrating.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
userrating_matrix = csr_matrix(userrating_pivot.values)


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(userrating_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [13]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  book_list_name = []
  book_id = userrating_pivot.index.get_loc(book)
  distances, indices = model_knn.kneighbors(userrating_pivot.iloc[book_id, :].values.reshape(1, -1), n_neighbors = 6)
  for i in range(0, len(distances.flatten())):
    if i == 0:
      book_list_name.append(userrating_pivot.index[book_id])
    else:
      book_list_name.append([userrating_pivot.index[indices.flatten()[i]], distances.flatten()[i]])


  return book_list_name

In [31]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False
    recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
    for i in range(1, 3): # Corrected loop range to start from 1
      print(recommends[i])
      if recommends[i][0] not in recommended_books:
          test_pass = False
      if abs(recommends[i][1] - recommended_books_dist[i-1]) >= 0.05: # Corrected distance comparison
          test_pass = False
    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

["Where the Heart Is (Oprah's Book Club (Paperback))", ['The Lovely Bones: A Novel', 0.90959823], ['Blue Diary', 0.9101138], ['I Know This Much Is True', 0.91014755], ["The Pilot's Wife : A Novel", 0.9172571], ['The Weight of Water', 0.9210188]]
